In [36]:
## Set up dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb

df = pd.read_csv("../data/youtube_project_database.csv")

In [37]:
## Clean up data

import re

#Views
df = df[df['viewCount'].notna()]
df['logViews'] = np.log(df['viewCount'] + 1)

mean_log_views = np.mean(df['logViews'])
std_log_views = np.std(df['logViews'])
df['zLogViews'] = (df['logViews'] - mean_log_views) / std_log_views

# Categories
df["categoryId"].astype("category")

# Average Views
df = df[df["videoCount"] > 0]
df['avgViewsPerVid'] = pd.to_numeric(df['avgViewsPerVid'])

# Titles
def process_title(title):
    title = str(title)
    # title = re.sub(r"&.*?;", " ", title)
    title = re.sub(r"[!\"#\＄%&\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]", " ", title)
    title = title.lower()
    title = title.split(" ")
    title = list(filter(None, title))
    return title

titles = df["vidTitle"].map(process_title)

# Tags
def process_tags(tags):
    if(tags == None):
        return []
    tags = str(tags)
    tags = tags.lower()
    tags = tags[1:-1]
    tags = tags.split(", ")
    tags = [tag[1:-1] for tag in tags]
    tags = list(filter(None, tags))
    return tags

tags = df["tags"].map(process_tags)
df["tagCount"] = len(tags)

# Times
df.loc[df['retrievalTime'].isna(), 'retrievalTime'] = '2023-04-07 14:25:00'

df['publishedAt'] = pd.to_datetime(df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ')
df['retrievalTime'] = pd.to_datetime(df['retrievalTime'], format='%Y-%m-%d %H:%M:%S')
df['timeElapsed'] = (df['retrievalTime'] - df['publishedAt']).dt.days

C:\Users\me123\AppData\Local\Temp\ipykernel_11572\1344976407.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['retrievalTime'].isna()]['retrievalTime'] = '2023-04-07 14:25:00'


In [101]:
## Train Model for Word2Vec

from gensim.models import Word2Vec, FastText

def convertToVec(w2v, wordlist):
    if (len(wordlist) == 0):
        return np.zeros(w2v.vector_size)
    else:
        return np.mean([w2v.wv[word] if (word in w2v.wv) else np.zeros(w2v.vector_size) for word in wordlist], axis=0)

vector_size_titles = 300
w2v_titles = Word2Vec(titles, min_count=2, vector_size = vector_size_titles)
title_vectors = [convertToVec(w2v_titles, title) for title in titles]
title_vec_cols = ["titleVec" + str(num) for num in range(vector_size_titles)]
title_vec_df = pd.DataFrame(title_vectors, index=df.index, columns = title_vec_cols)
df = pd.concat([df, title_vec_df], axis=1)

vector_size_tags = 300
w2v_tags = Word2Vec(tags, min_count=2, vector_size = vector_size_tags)
tags_vectors = [convertToVec(w2v_tags, tagList) for tagList in tags]
tags_vec_cols = ["tagsVec" + str(num) for num in range(vector_size_tags)]
tags_vec_df = pd.DataFrame(tags_vectors, index=df.index, columns = tags_vec_cols)
df = pd.concat([df, tags_vec_df], axis=1)

In [106]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

x_labels = ["categoryId", "tagCount", "avgViewsPerVid", "subscriberCount", "videoCount", "timeElapsed"] + title_vec_cols + tags_vec_cols
y_labels = ["zLogViews"]
x = df.loc[:, x_labels]
y = df.loc[:, y_labels]
dmatrix = xgb.DMatrix(data=x,label=y, enable_categorical=True)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                           learning_rate = 0.1,
                           max_depth = 3,
                           n_estimators = 200)

xgb_reg.fit(x_train, y_train)
pred = xgb_reg.predict(x_test)

print("MAE (just mean): " + str(mean_absolute_error(np.exp(y_test), x_test["avgViewsPerVid"])))
print("MAE (all params): " + str(mean_absolute_error(np.exp(y_test * std_log_views + mean_log_views), np.exp(pred * std_log_views + mean_log_views))))

MAE (just mean): 1813171.0902219412
MAE (all params): 521721.8544758588
